In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from app.config import get_llm
import pandas as pd

In [8]:
llm = get_llm()

In [14]:
parser = StrOutputParser()

In [18]:
test_queries = [
    "I was charged twice for my last invoice.",
    "My password reset link doesn’t work.",
    "Can I change my email address for notifications?",
    "Nie mogę zalogować się na moje konto mimo poprawnego hasła.",
    "Please cancel my subscription immediately.",
]

In [19]:
prompts = {
    "baseline": PromptTemplate.from_template(
        "Classify the following customer message into one of these categories: "
        "Billing, Account, Technical Issue, or Other.\n\n"
        "Message: {query}\n"
        "Answer:"
    ),
    "json_schema": PromptTemplate.from_template(
        "You are a customer support ticket classifier. "
        "Analyze the message and respond ONLY as valid JSON:\n"
        "{{\"category\": \"Billing|Account|Technical|Other\", \"confidence\": <0-1>}}\n\n"
        "Message: {query}"
    ),
    "few_shot": PromptTemplate.from_template(
        "You are a support classifier. Based on examples:\n"
        "- 'I was charged twice' → Billing\n"
        "- 'I forgot my password' → Account\n"
        "- 'My app crashes' → Technical Issue\n\n"
        "Now classify this message:\n{query}\nAnswer:"
    ),
}

In [20]:
results = []

In [23]:
for name, prompt in prompts.items():
    chain = prompt | llm | parser
    for q in test_queries:
        try:
            resp = chain.invoke({"query": q})
        except Exception as e:
            resp = f"Error: {e}"
        results.append({
            "prompt_variant": name,
            "query": q,
            "response": resp
        })

In [24]:
df_results = pd.DataFrame(results, columns=["prompt_variant", "query", "response"])

In [25]:
print("shape:", df_results.shape)
print("cols:", df_results.columns.tolist())
display(df_results.head())

shape: (15, 3)
cols: ['prompt_variant', 'query', 'response']


,prompt_variant,query,response
0,baseline,I was charged twice for my last invoice.,Billing. This message indicates that the cust...
1,baseline,My password reset link doesn’t work.,Technical Issue.
2,baseline,Can I change my email address for notifications?,Category: Account.
3,baseline,Nie mogę zalogować się na moje konto mimo popr...,Technical Issue (Account-related)\n\nExplanat...
4,baseline,Please cancel my subscription immediately.,"Based on the given message, it can be classif..."
